# Trabajo Final: 

Generación automática de consultas SQL y respuestas ad hoc sobre un ecosistema de datos.
El objetivo del proyecto es dar solución a una problemática que sucede en las empresas con los modelos de datos y su entendimiento. Para los usuarios experimentados es terreno conocido, pero para los usuarios nuevos la documentación suele ser poco amigable y la curva de aprendizaje se extiende en el tiempo. Este proyecto está orientado a acortar esta curva de aprendizaje y a acercar la información de las tablas a las personas y sus consultas.

## Índice
1. Introducción
2. Objetivos
3. Metodología
4. Herramientas y Tecnologías
5. Implementación
6. Resultados
7. Conclusiones
8. Referencias

## 1. Introducción
En el contexto empresarial, un desafío común al que se enfrentan los gerentes o jefes de Datos es la dificultad de transmitir de manera eficiente el conocimiento sobre el modelo de datos de la organización a nuevos analistas, ya sea recién ingresados al equipo o internos que necesitan familiarizarse con el esquema de datos.
Los modelos de datos suelen ser extensos y complejos, con nomenclaturas poco intuitivas que dificultan su comprensión inicial. Esta curva de aprendizaje puede extenderse por meses, ralentizando la productividad de los analistas y generando ineficiencias en la operación.
Acelerar el entendimiento del modelo de datos, la construcción de consultas para obtener información específica y el conocimiento sobre la composición de campos, filas y calidad de la información representaría un valor significativo para la organización.

## 2. Objetivos
El objetivo principal es acelerar el entendimiento de un modelo de datos y facilitar el acceso a información clave. Aplica a perfiles técnicos que precisen conocer el modelo y perfeccionar sus querys aplicándolas a casuísticas reales, o bien a usuario no técnicos que tienen preguntas concretas sobre la información de la empresa y que quieren entender las respuestas y también su vía de obtención.

## 3. Metodología
- Conexión con la base de datos: Conexiones vía Python a un set de datos.
- Creación de las consultas SQL: Utilización de GPT 3.5 para el entendimiento de la pregunta del requirente y su traducción a SQL.
- Conversión a lenguaje natural: Utilización de GPT 3.5 para la traducción del resultado del SQL a un lenguaje natural coloquial.
- Iteración: Funciones de contexto acumulado y preguntas encadenadas para lograr iteración y repreguntas.

## 4. Herramientas y Tecnologías
- Kernel: Python
- AI: gpt-3.5-turbo
- Librerías utilizadas: psycopg2 openai pandas


In [18]:
# Librerías necesarias
from psycopg2 import connect
import openai
import pandas as pd

In [ ]:
# Configuración del modelo de lenguaje
openai.api_key = 'xxxxx'

In [ ]:
# 1. Configurar conexión a Redshift
def conectar_redshift():
    """
    Crea y devuelve una conexión a Redshift.
    """
    return connect(
    dbname='xxxxx',
    user='xxxxx',
    password='xxxxx',
    host='xxxxx',
    port='xxxxx'
)

In [21]:
 # Lista global para almacenar variables
contexto_acumulado = [] 

pregunta_usuario = []

pregunta_siguiente = []

sql_generado = []

In [22]:
# Contexto detallado del promt inicial
contexto = """
            Usa tablas relevantes según la pregunta. Respeta siempre las convenciones de los roles específicos.Eres un experto en bases de datos SQL, especializado en sistemas complejos como Redshift. 
            Tu tarea es convertir preguntas de usuarios no técnicos en consultas SQL eficientes y optimizadas, 
            teniendo en cuenta el esquema y las relaciones entre tablas, así como las mejores prácticas en SQL. Debes tener en cuenta las siguientes reglas:

            Posibles paises en los que tenemos operacion: Argentina y Colombia.

            Para consultas sobre Colombia, usa las tablas que terminan en _col. adicionalmente los schemas sobre los que hacer las consultas tambein terminan en _col.
            Para consultas sobre Argentina, usa las tablas que terminan en _arg. adicionalmente los schemas sobre los que hacer las consultas tambein terminan en _arg.

            Para entender las conexiones entre las tablas el concepto principal es que las tablas tienen un ID y se vinculan con otra tabla de referencia. Por ejemplo, la tabla LEADS tiene un ID, y todas las tablas que tengan el campo LEAD_ID joinean contra ella. 
            
            Las tablas relevantes para entender de las ventas de los creditos son:

            Argentina: core_arg.loans_arg
            Colombia: core_col.loans_col
            
            En donde los campos mas relevantes para entender los requerimientos de los usuarios son diferenes para cada pais, en el caso de Argentina son:

            date(payment_date): es la fecha de venta o desembolso del credito. Siempre debemos ponerle la funcion date() previamente para convertirlo en fecha.
            date(created_at): es la fecha de creacion del registro en la tabla. Siempre debemos ponerle la funcion date() previamente para convertirlo en fecha.
            is_renovation: true o 1 indica si el cliente es renovador y false o 0 cuando es nuevo.
            term: es el plazo del credito.
            amount: es el monto o capital vendido
            installment_amount: es el monto que tiene que pagar el cliente por mes, es decir la cuota.
            el interes del credito puede ser calculado multiplicando el installment_amount * el term y restandole a ello el amount.
            rate: es un dato de clasificacion importante que se llama rate.
            lead_id: es un campo que permite el join contra otra tabla importante core_arg.leads_arg via el campo core_arg.leads_arg.id
            
            Es importante para entender la venta de argentina que solo hay que tomar los registros que tengan status_id in (20,39), el resto son ventas anuladas.

            En el caso de colombia los campos relevantes son similares, perolevemente diferentes:

            date(transferred_date): es la fecha de venta o desembolso del credito. Siempre debemos ponerle la funcion date() previamente para convertirlo en fecha.
            date(created_at): es la fecha de creacion del registro en la tabla.Siempre debemos ponerle la funcion date() previamente para convertirlo en fecha.
            is_renovation: true o 1 indica si el cliente es renovador y false o 0 cuando es nuevo.
            terms: es el plazo del credito.
            total_amount: es el monto o capital vendido
            installment_amount: es el interes total que paga el cliente.
            rate: es un dato de clasificacion importante que se llama rate.
            lead_id: es un campo que permite el join contra otra tabla importante core_arg.leads_arg via el campo core_col.leads_col.id

            Es importante para entender la venta de colombia que solo hay que tomar los registros que tengan status_id in (22,24), el resto son ventas anuladas.
           
            Por otro lado, las tablas relevantes de cobranzas son las siguientes:
            Argentina: collections_arg.cobranza_arg
            Colombia: collections_col.cobranza_col
            
            Por otro lado, las tablas relevantes a "Pegadas del motor" o "consutlas del motor" o "Consultas de riesgos" o "consutlas de uflow" son las siguientes:
            
            Argentina: gold.consultas_motor_arg
            Esta tablas se pueden unir contra la tabla leads mediante el ik id y partiendo de la columna external_id cuando el external_type = 'lead_id', este join solo debe hacerse si se quiere obtener informacion adicional, sino no es necesario para pbtener data exclusiva de las consultas al motor de riesgo.
            En esta tabla el ID unico es: executionid
            El campo de fecha de consulta es: executiondate_date
            La decision del motor, el estado de la consulta si es aprobado, rechazado o error,  la obtenes con el siguiente campo: decisionresult_lite este es el estado de la consulta, la resolucion del motor. 
            la fuente se obtiene del campo: Fuente
            El nivel de riesgo se obtiene del campo: perfil_riesgo
            El nro de documento de las personas se encuentra en el campo: nrodoc.
            
-- 
            Colombia: risk_col.risk_engine_col
            Esta tablas se pueden unir contra la tabla leads mediante el ik id y partiendo de la columna lead_id. , este join solo debe hacerse si se quiere obtener informacion adicional, sino no es necesario para pbtener data exclusiva de las consultas al motor de riesgo.
            En esta tabla el ID unico es: executionid
            El campo de fecha de consulta es: date(executiondate)
            La decision del motor la obtenes con el siguiente case: 'case 
                                                                        when upper(split_part(rec.decisionresult,' ','1')) = 'LEAD' then 'APROBADO'
                                                                        when upper(split_part(rec.decisionresult,' ','1')) = 'APROBADO' then 'APROBADO'
                                                                        when upper(split_part(rec.decisionresult,' ','1')) = 'RECHAZO' then 'RECHAZADO'
                                                                        when upper(split_part(rec.decisionresult,' ','1')) = 'RECHAZADO' then 'RECHAZADO'
                                                                        when upper(split_part(rec.decisionresult,' ','1')) in ('ERROR', 'NO') then 'ERROR'
                                                                        when rec.decisionresult is null or rec.decisionresult = '' then 'DECISION_EN_BLANCO'
                                                                        else 'INVESTIGAR' end estado_consulta'
            la fuente se obtiene del campo: Fuente
            El nivel de riesgo se obtiene del campo: perfil_riesgo
            El nro de documento de las personas se encuentra en el campo: Cedula.
            
            Si se dean obtener datos de columnas referentes a 'mareigua' el join con las tablas es el siguiente:
            left join risk_col.risk_engine_2_col rec2 
            on risk_col.risk_engine_col.executionid = rec2.executionid_2        
            
            A nivel general y para todas las consultas de SQL a generar, para una mejor performance NO utilizar DATE_TRUNC, mejor usar EXTRACT o DATE_PART o bien between de fechas.
            Tambien es importante que a todos los campos de FECHAS antes le agreguemos la funcion date(), porque muchos de ellos tienen el formato equivocado.
            Por otro lado como mejor practica, NUNCA utilices en los group by el alias del campo, siempre utiliza el campo en si, ya sea tal como existe, o la construcciond el campo en si mismo.
            
            Asegúrate de generar consultas seguras y optimizadas, respetando el contexto geográfico de las tablas y las relaciones de la base de datos.
            
            Lo mas importante, cuando entregues el resultado, solo entrega la consulta SQL en sintaxis de AWS redshift, debes validar que la sintaxis de la query sea correcta y que no falle su ejecucion, 
            ademas, no agregues redaccion al resultado, ya que con tu consulta de SQL luego voy a ejecutar en base de datos para obtener el resultado.
            
            Convierte la siguiente pregunta en una consulta SQL:
                """

In [23]:
contexto_acumulado.append(contexto)

In [24]:
def generar_sql_con_ia(pregunta_usuario, contexto_acumulado):
    """
    Utiliza OpenAI para generar una consulta SQL basada en la pregunta del usuario.
    Se adapta según el rol del usuario, utilizando solo tablas y campos relevantes definidos.
    """
    conversation = [
        {
            "role": "system",
            "content": f"""
            Eres un asistente experto en SQL para bases de datos Amazon Redshift. Genera consultas SQL basadas en preguntas del usuario, usando exclusivamente las tablas y campos proporcionados. 
            Contexto del sistema: {contexto_acumulado}
            Devuelve solo la consulta SQL, sin explicaciones adicionales.
            """
        },
        {"role": "user", "content": f"Pregunta: {pregunta_usuario}\n\nConsulta SQL:"}
    ]

    try:
        # Llamada al modelo de OpenAI
        respuesta = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  
            messages=conversation,
            max_tokens=1000,
            temperature=0.5,
        )

        # Limpiar la salida del modelo
        sql_generado = respuesta.choices[0].message['content'].strip()

        if sql_generado.startswith("```sql"):
            sql_generado = sql_generado[6:]  # Eliminar ```sql
        if sql_generado.endswith("```"):
            sql_generado = sql_generado[:-3]  # Eliminar ```
        
        # Remover saltos de línea
        sql_generado = sql_generado.replace("\n", " ")


        # Actualizar el contexto acumulativo
        contexto_acumulado.append({"pregunta": pregunta_usuario, "sql": sql_generado})

        return sql_generado.strip()

    except Exception as e:
        print(f"Error al generar SQL: {e}")
        return None


In [25]:
def ejecutar_consulta_redshift(sql_generado):
    """
    Ejecuta una consulta SQL en Amazon Redshift y devuelve los resultados.
    """
    try:
        conexion = conectar_redshift()
        with conexion.cursor() as cursor:
            cursor.execute(sql_generado)
            columnas = [desc[0] for desc in cursor.description]  # Obtener nombres de columnas
            resultados = [dict(zip(columnas, fila)) for fila in cursor.fetchall()]
            
        conexion.close()
        return resultados, contexto_acumulado.append(resultados)

    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
        return None

In [26]:
def transformar_a_lenguaje_natural(resultados, pregunta_usuario, contexto_acumulado):

    if not resultados:
        return "No se encontraron resultados para tu consulta."
    
    try:
        conversation = [
            {"role": "system", "content": "Eres un asistente que convierte datos tabulares en respuestas claras y amigables. tu rol principal es explicar los resultados a una persona que quiere entender lo que ha preguntado"},
            {"role": "user", "content": f"""
            Contexto: La consulta SQL devolvió estos resultados: {resultados}.
            Responde de manera profesional y clara basándote en la pregunta: "{pregunta_usuario}".
            """}
        ]
        respuesta = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages=conversation,
            max_tokens = 1000,
            temperature = 0.5,
        )
        
                # Actualizar el contexto acumulativo con la respuesta
        contexto_acumulado.append(respuesta.choices[0].message['content'].strip())
        
        return respuesta.choices[0].message['content'].strip()
    except Exception as e:
        print(f"Error al transformar resultados: {e}")
        return "Hubo un error al generar la respuesta en lenguaje natural."

In [27]:
# 5. Función principal para procesar la pregunta
def procesar_pregunta(pregunta_usuario):
    """
    Procesa la pregunta del usuario y devuelve una respuesta en lenguaje natural.
    """
    # Paso 1: Generar la consulta SQL
    consulta_sql = generar_sql_con_ia(pregunta_usuario, contexto_acumulado=contexto_acumulado)
    if not consulta_sql:
        return "No entendí la consulta, ¿Podés reformularla?"
    
    # Imprimir la consulta SQL generada
    print(f"Gracias por tu consulta: * {pregunta_usuario} * \n. Generé la siguiente consulta SQL con la cual la voy a responder:\n {consulta_sql}")
    
    # Paso 2: Ejecutar la consulta en Redshift
    resultados = ejecutar_consulta_redshift(consulta_sql)
    if not resultados:
        return "Hubo un error al ejecutar la consulta en Redshift."
    
    # Paso 3: Transformar los resultados en una respuesta coloquial
    respuesta = transformar_a_lenguaje_natural(resultados, pregunta_usuario, contexto_acumulado=contexto_acumulado)
    
    print(respuesta)
    
    return respuesta
    

In [28]:
def consulta_integrada():
    pregunta_usuario = input("¿Qué te interesa consultar sobre nuestra empresa? ")
    procesar_pregunta(pregunta_usuario)
    
    contexto_acumulado.append(pregunta_usuario)
    

    

In [29]:
def consulta_siguiente():
    pregunta_siguiente = input("¿Qué te interesa saber? ")
    procesar_pregunta(pregunta_siguiente)
    
    contexto_acumulado.append(pregunta_siguiente)

In [30]:
def consulta_encadenada():
    while True:
        resultado = consulta_integrada()
        
        if resultado != "Hubo un error al ejecutar la consulta en Redshift.":
            
            
            while True:
                continuar = input("¿Deseás realizar otra consulta? (si/no): ").strip().upper()
                if continuar == 'SI':
                    consulta_siguiente()
                    
                else:
                    print("Deteniendo la ejecución de la consulta.")    
                    break
            
            break
        
        else:
            
            reformular = input("Hubo un error al ejecutar la consulta. ¿Desea reformularla? (si/no): ").strip().lower()
            if reformular.upper != 'SI':
                print("Finalizando el proceso.")
                break


In [31]:
if __name__ == "__main__":
    consulta_encadenada()

Gracias por tu consulta: * ¿Cuanto vendimos en argentina en el 2024 en cantidad y monto por mes? * 
. Generé la siguiente consulta SQL con la cual la voy a responder:
 SELECT DATE_TRUNC('month', date(payment_date)) AS month,        COUNT(*) AS quantity,        SUM(amount) AS total_amount FROM core_arg.loans_arg WHERE date(payment_date) >= '2024-01-01'   AND date(payment_date) < '2025-01-01'   AND status_id IN (20, 39) GROUP BY DATE_TRUNC('month', date(payment_date)) ORDER BY DATE_TRUNC('month', date(payment_date));
Según los resultados de la consulta SQL, en Argentina en el año 2024 se vendió la siguiente cantidad y monto por mes:

- Enero: Cantidad vendida: 1,759 unidades, Monto total: $130,446,544.00
- Febrero: Cantidad vendida: 1,940 unidades, Monto total: $175,047,736.00
- Marzo: Cantidad vendida: 1,704 unidades, Monto total: $175,066,000.00
- Abril: Cantidad vendida: 1,464 unidades, Monto total: $155,185,000.00
- Mayo: Cantidad vendida: 1,601 unidades, Monto total: $188,605,000.00

## 6. Resultados
En los resultados se puede ver como el usuario accede a resultados explicados, pero también a la forma de extraer esta información de la base. Esto puede resultar muy útil ya que no solo "Da de comer" sino que también "Enseña a pescar".

## 7. Conclusiones
Con un contexto inicial robusto, que esté alineado a la evolución del set de datos de la empresa, esta herramienta podría ser la primera aproximación a la información de una unidad de negocios y luego convertirse en un pívot entre la data y las preguntas.

## 8. Referencias
En este ejemplo se consume datos de un set de datos de AWS(Redshift) pero las aplicaciones son infinitas.
